In [2]:
import sys
sys.path.append("/Users/alex/Downloads/stats/")

import pandas as pd
import os
from db import connection
from db_utils import DBUtils
import xlrd
from tqdm import tqdm
import datetime
import json
from glob import glob

In [3]:
values = ['UN WPP - Interpolated demographic indicators by region, subregion and country, annually for 1950-2099',
 'UN WPP - Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
 'UN WPP - Interpolated male population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
 'UN WPP - Interpolated female population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
 'UN WPP - Percentage of total population by broad age group, region, subregion and country, annually interpolated for 1950-2100',
 'UN WPP - Dependency ratios (total, child, old-age) for different age groups and for both sexes combined by region, subregion and country, annually interpolated for 1950-2100',
 'UN WPP - Total population (both sexes combined) by single age, region, subregion and country, annually for 1950-2100 (thousands)']

keys = ['Annually interpolated demographic indicators.', 'Annual total population (both sexes combined) by broad age groups. Data are presented in thousands.', 'Annual male population by broad age groups. Data are presented in thousands.', 'Annual female population by broad age groups. Data are presented in thousands.', 'Percentage of annual total population (both sexes combined) by broad age group.', 'Annual dependency ratios (total, child, old-age) for different age groups and for both sexes combined.', 'Annual population by single age - Both Sexes. De facto population as of 1 July of the year indicated classified by single age (0, 1, 2, ..., 99, 100+). Data are presented in thousands.']

datasets_dict = {}
for x,y in zip(keys, values):
    datasets_dict[x] = y
    

In [4]:
# with connection as c:
#     db = DBUtils(c)
    
#     # upsert datasets
#     dataset_name_ids = {}
#     for f in tqdm(os.listdir('sources/')):
#         if f == '.DS_Store':
#             continue
#         data = pd.read_excel('sources/'+f)
#         val = data[data.columns[0]][8]
#         index_to_remove = val.find(":")
#         res = "UN WPP - " + val[index_to_remove+2:]
#         dataset_id = db.upsert_dataset(name=res, namespace="unwpp", user_id=15)
#         dataset_name_ids[res] = dataset_id
        
#     # upsert sources
    
#     dataset_to_source_ids = {}
#     source_name = "United Nations – Population Division (2019 Revision)"
#     for addInfo, dataset_name in datasets_dict.items():
#         description = {}
#         description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
#         description['dataPublisherSource'] = None
#         description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
#         description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
#         description['additionalInfo'] = addInfo
        
#         source_id = db.upsert_source(name=source_name, description=json.dumps(description), dataset_id=dataset_name_ids[dataset_name])
#         dataset_to_source_ids[dataset_name_ids[dataset_name]] = source_id
    

In [5]:
# dataset_to_source_ids

# Preparing csvs

## Datasets

In [7]:
names,ids = [], []
i = 0
for f in tqdm(glob('data/*.xlsx')):
    
        data = pd.read_excel(f)
        val = data[data.columns[0]][8]
        index_to_remove = val.find(":")
        res = "UN WPP - " + val[index_to_remove+2:]
        names.append(res)
        ids.append(i)
        i+=1
datasets = pd.DataFrame()
datasets['id'] = ids
datasets['name'] = names
datasets.to_csv('datasets.csv', index=False)

100%|██████████| 7/7 [02:39<00:00, 20.84s/it]


## Sources

In [8]:
names, desc, d_ids = [], [], []

source_name = "United Nations – Population Division (2019 Revision)"
for addInfo, dataset_name in datasets_dict.items():
    description = {}
    description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
    description['dataPublisherSource'] = None
    description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
    description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
    description['additionalInfo'] = addInfo
    dataset_id = datasets[datasets['name'] == dataset_name]['id'].values[0]

    names.append(source_name)
    desc.append(description)
    d_ids.append(dataset_id)
    
res = pd.DataFrame()
res['name'] = names
res['description'] = desc
res['dataset_id'] = d_ids
res.to_csv("sources.csv", index=False)

## Variables

In [30]:
class DataVariables():
    
    def __init__(self):
        
        
        self.i = 0
        self.ids = []
        self.names = []
        self.units = []
        self.dataset_ids = []
        self.doc_to_unit = {
                        "UN WPP - Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
                        'UN WPP - Total population (both sexes combined) by single age, region, subregion and country, annually for 1950-2100 (thousands)': "Thousands",
                        'UN WPP - Percentage of total population by broad age group, region, subregion and country, annually interpolated for 1950-2100': "Percentage",
                        'UN WPP - Interpolated male population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)': 'Thousands',
            'UN WPP - Interpolated female population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)': 'Thousands',
             'UN WPP - Dependency ratios (total, child, old-age) for different age groups and for both sexes combined by region, subregion and country, annually interpolated for 1950-2100': 'Percentage'

            #'UN WPP - Interpolated demographic indicators by region, subregion and country, annually for 1950-2099': 
                        

                    }
        
        self.datasets = pd.read_csv('datasets.csv')
        self.df = pd.DataFrame()
    
        
    def get_variables(self, path, skiprows=8, prefix=None):
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            print(title)

            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                if prefix:
                    self.names.append(title + ": " + prefix + " - " + item)
                else:
                    self.names.append(title + ": " + item)
                self.ids.append(self.i)
                self.i+=1
                self.units.append(self.doc_to_unit[res])
                self.dataset_ids.append(self.datasets[self.datasets['name'] == res]['id'].values[0])
    
    def get_custom_variable(self, path, skiprows=8, prefix=None):
        
        column_unit = {
            8: "Thousands",
            9: "Thousands",
            10: "Thousands",
            11: "Percentage",
            12: "Years",
            13: "Years",
            14: "Years",
            15: "Thousands",
            16: "Infant deaths per 1,000 live births",
            17: "Deaths under age 5 per 1,000 live births",
            18: "Thousands",
            19: "Births per 1,000 population",
            20: "Live births per woman",
            21: "Thousands",
            22: "Per 1,000 population",
            23: "Thousands",
            24: "Percentage"
            
        
        }
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            print(title)
            
            col_index = 8
            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                if prefix:
                    self.names.append(title + ": " + prefix + " - " + item)
                else:
                    self.names.append(title + ": " + item)
                self.ids.append(self.i)
                self.i+=1
                self.units.append(column_unit[col_index])
                self.dataset_ids.append(self.datasets[self.datasets['name'] == res]['id'].values[0])
                col_index += 1
           
    

            
            
    def get_df(self):
        
        self.df['id'] = self.ids
        self.df['name'] = self.names
        self.df['unit'] = self.units
        self.df['dataset_id'] = self.dataset_ids
        
        return self.df
        
        
        

In [31]:
dv = DataVariables()
for f in tqdm(glob('data/*.xlsx')):
     
        if f == "data/unwpp.xlsx":
            dv.get_variables(f, prefix="Total population by broad age group, both sexes combined (thousands)")
        elif f == "data/unwpp_total_population.xlsx":
            dv.get_variables(f, prefix="Total population by age, both sexes combined (thousands)")
        elif f == "data/unwpp_percentage_of_total_population.xlsx":
            dv.get_variables(f, prefix="Percentage of total population by broad age group, both sexes combined (per 100 total population)")
        elif f == "data/unwpp_male_population.xlsx":
            dv.get_variables(f, prefix="Male population by broad age group (thousands)")
        elif f == "data/unwpp_interpolated_demographic_indicators.xlsx":
            dv.get_custom_variable(f, prefix="Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)")
        elif f == "data/unwpp_female_population.xlsx":
            dv.get_variables(f, prefix="Female population by broad age group (thousands)")
        elif f == "data/unwpp_dependency_ratios.xlsx":
            dv.get_variables(f, prefix="Dependency ratios (both sexes combined) for different age groups")
        

res = dv.get_df()


  0%|          | 0/7 [00:00<?, ?it/s]

Estimates, 1950 - 2020


 14%|█▍        | 1/7 [00:47<04:44, 47.47s/it]

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020


 29%|██▊       | 2/7 [01:33<03:55, 47.15s/it]

...

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020


 86%|████████▌ | 6/7 [04:57<00:51, 51.39s/it]

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020


100%|██████████| 7/7 [05:18<00:00, 42.37s/it]

Medium fertility variant, 2020 - 2100


In [34]:
res.to_csv("variables.csv", index=False)

In [33]:
res[res['unit'] == 'Per 1,000 population']

,id,name,unit,dataset_id
242,242,"Estimates, 1950 - 2020: Interpolated total pop...","Per 1,000 population",2
259,259,"Medium fertility variant, 2020 - 2099: Interpo...","Per 1,000 population",2


## Datapoints

In [84]:
def normalize_country(row):
        
        row['country'] = row['country'].str.replace(r'\s*[^A-Za-z\s]*$', '')
        return row


def get_variables(path, variables, skiprows=8, prefix=None, custom=False):
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            
            index_col = 8
            
            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                
                if prefix:
                    var_name = title + ": " + prefix + " - " + item
                else:
                    var_name = title + ": " + item
                
                var_id = variables[variables['name'] == var_name]['id'].values[0] 
                
                data2 = data.iloc[8:]
               
                data_res = pd.DataFrame()
                data_res['country'] = data2[data2.columns[2]]
                data_res['year'] = data2[data2.columns[7]]
                data_res['value'] = data2["Unnamed: %s" % str(index_col)]
                
                data_res['country'] = normalize_country(data_res)
                data_res.to_csv('datapoints/datapoints_%s.csv' % str(var_id), index=False)
                
                index_col += 1
            

In [86]:
variables = pd.read_csv("variables.csv")

for f in tqdm(glob('data/*.xlsx')):
   

    if f == "data/unwpp.xlsx":
        get_variables(f, variables, prefix="Total population by broad age group, both sexes combined (thousands)")
    elif f == "data/unwpp_total_population.xlsx":
        get_variables(f,variables, prefix="Total population by age, both sexes combined (thousands)")
    elif f == "data/unwpp_percentage_of_total_population.xlsx":
        get_variables(f,variables, prefix="Percentage of total population by broad age group, both sexes combined (per 100 total population)")
    elif f == "data/unwpp_male_population.xlsx":
        get_variables(f,variables, prefix="Male population by broad age group (thousands)")
    elif f == "data/unwpp_interpolated_demographic_indicators.xlsx":
        get_variables(f,variables, prefix="Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)")
    elif f == "data/unwpp_female_population.xlsx":
        get_variables(f,variables, prefix="Female population by broad age group (thousands)")
    elif f == "data/unwpp_dependency_ratios.xlsx":
        get_variables(f,variables, prefix="Dependency ratios (both sexes combined) for different age groups")























  0%|          | 0/8 [00:00<?, ?it/s]




















 12%|█▎        | 1/8 [00:55<06:29, 55.70s/it]




















 38%|███▊      | 3/8 [01:51<03:56, 47.32s/it]




















 50%|█████     | 4/8 [02:11<02:36, 39.21s/it]




















 62%|██████▎   | 5/8 [03:43<02:45, 55.15s/it]




















 75%|███████▌  | 6/8 [04:37<01:49, 54.85s/it]




















 88%|████████▊ | 7/8 [05:33<00:55, 55.05s/it]




















100%|██████████| 8/8 [05:56<00:00, 45.40s/it]